Organization:
- Work
  - 1 test: defining functions for part 1, testing on test input
  - 1 run: getting answer for part 1
  - 2 test: ...
  - 2 run: ...
- Utilities: functions I think might help parse general inputs
- Inputs: where I define the test (_t_) and problem (_s_) inputs

# Work

## 1 test

Thoughts:
- Solving the problem iteratively for 1, 2, ..., 30 won't work: possible ex. the optimal first step if you have 2 minutes is to open a nearby valve, but if you have 30 minutes it's to move a few steps farther to a big valve, since any benefit from the small valve is beat by the big valve being open for 1 minute longer
- For 10 valves there's 10! < 4,000,000 orders to open valves, so it's somewhat feasible to check every order of opening valves using (precomputed) graph shortest paths to figure out when each valve is opened. But this would not work for 61 valves.
- This general method should work though
  - DFS on the tree of n! paths, using precomputed graph shortest paths for travel times
  - To open a valve costs time to get there + 1 and you "earn" its flow rate * minutes left in value from it
  - Keep a list of the valves not yet visited (remove valves that have 0 flow; this means there's effectively on 14 valves for the main problem)
  - Record the current value of the actions taken (in amount of pressure they'll have relieved by the end of the 30 minutes) and an overestimate for the maximum value possible (given by the sum of unopened flows * the time left)
  - After each step (before branching), abort if the surrogate max value is below the best value found so far
  - If we're at the end of the path, possibly update the best total flow

In [105]:
# A bit hacky, just parsing a single line of the input
def parse(line):
    aux = line.split(' ')
    me = aux[1]
    flow = int(aux[4].split('=')[1].split(';')[0])
    others = [other.strip(',') for other in aux[9:]]
    return (me, flow, others)

In [106]:
valves = [parse(line) for line in split(t)]
valves

[('AA', 0, ['DD', 'II', 'BB']),
 ('BB', 13, ['CC', 'AA']),
 ('CC', 2, ['DD', 'BB']),
 ('DD', 20, ['CC', 'AA', 'EE']),
 ('EE', 3, ['FF', 'DD']),
 ('FF', 0, ['EE', 'GG']),
 ('GG', 0, ['FF', 'HH']),
 ('HH', 22, ['GG']),
 ('II', 0, ['AA', 'JJ']),
 ('JJ', 21, ['II'])]

In [107]:
vertices = {}
names = {}
for i, valve in zip(range(len(valves)), valves):
    vertices[valve[0]] = i
    names[i] = valve[0]
vertices

{'AA': 0,
 'BB': 1,
 'CC': 2,
 'DD': 3,
 'EE': 4,
 'FF': 5,
 'GG': 6,
 'HH': 7,
 'II': 8,
 'JJ': 9}

In [108]:
import numpy as np

In [109]:
# Form the adjacency matrix and check for symmetry and no diagonal
# Also did this for the 2nd input and the checks also passed
n = len(valves)
A = np.zeros((n,n), dtype=int)
for me, flow, others in valves:
    for other in others:
        A[vertices[me], vertices[other]] = 1
A

array([[0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [110]:
(A == A.transpose()).all()

True

In [111]:
from scipy.sparse.csgraph import shortest_path

In [112]:
D = shortest_path(A).astype(int)
D

array([[0, 1, 2, 1, 2, 3, 4, 5, 1, 2],
       [1, 0, 1, 2, 3, 4, 5, 6, 2, 3],
       [2, 1, 0, 1, 2, 3, 4, 5, 3, 4],
       [1, 2, 1, 0, 1, 2, 3, 4, 2, 3],
       [2, 3, 2, 1, 0, 1, 2, 3, 3, 4],
       [3, 4, 3, 2, 1, 0, 1, 2, 4, 5],
       [4, 5, 4, 3, 2, 1, 0, 1, 5, 6],
       [5, 6, 5, 4, 3, 2, 1, 0, 6, 7],
       [1, 2, 3, 2, 3, 4, 5, 6, 0, 1],
       [2, 3, 4, 3, 4, 5, 6, 7, 1, 0]])

In [113]:
flows = [flow for me, flow, others in valves]
flows

[0, 13, 2, 20, 3, 0, 0, 22, 0, 21]

In [114]:
can_open = [i for i in range(n) if flows[i] > 0]
can_open

[1, 2, 3, 4, 7, 9]

In [116]:
max_value = 0

# Launch a DFS from vertex i with t minutes left
# value gives the value of the actions performed so far
# can_open gives valves we can still open
def DFS(i, t, value, can_open, history):
    global max_value, max_history
    
    # Iterate over possible valves to open
    for j in can_open:
        # If we have enough time to get to and open this valve and have it open at least 1 minute, try opening it
        time_to_open = D[i,j] + 1
        if time_to_open < t:
            # Parameters for the new DFS node
            new_t = t - time_to_open
            new_value = value + flows[j] * new_t
            new_can_open = [k for k in can_open if k != j]
            
            # Pruning: check if we already cannot achieve the max_value found so far
            new_total_value_est = new_value + sum([flows[k] for k in new_can_open]) * (new_t - 2)
            if new_total_value_est > max_value:
                # Continue DFS
                DFS(j, new_t, new_value, new_can_open, history + [names[j]])
    
    # Once we get here, we've tried opening all valves worth opening
    # This also includes the case where there were no valves to open or time ran out
    # So check if we achieved a new record value
    if value > max_value:
        max_value = value
        max_history = history

DFS(vertices['AA'], 30, 0, can_open, [])
print(max_value)
print(max_history)

1651
['DD', 'BB', 'JJ', 'HH', 'EE', 'CC']


## 1 run

In [117]:
valves = [parse(line) for line in split(s)]

vertices = {}
names = {}
for i, valve in zip(range(len(valves)), valves):
    vertices[valve[0]] = i
    names[i] = valve[0]

n = len(valves)
A = np.zeros((n,n), dtype=int)
for me, flow, others in valves:
    for other in others:
        A[vertices[me], vertices[other]] = 1

D = shortest_path(A).astype(int)

flows = [flow for me, flow, others in valves]

can_open = [i for i in range(n) if flows[i] > 0]

In [119]:
max_value = 0
DFS(vertices['AA'], 30, 0, can_open, [])
print(max_value)
print(max_history)

1857
['MA', 'II', 'AS', 'RU', 'PM', 'KQ', 'ED']


## 2 test

- With the elephant, movements won't be in sync: I might move to a vertex to open a valve, but the elephant would be on its way to another vertex to open a valve
- One solution: for each step move either me or the elephant to a valve, and the other goes to any other vertex. WLOG everyone is constantly moving until all time runs out so this doesn't mess up on any weird edge cases.
- Another one: plan my route first, and once that's done I'll plan the elephant's route (making sure it doesn't flip any valves I flipped)
- I opted for the 2nd one, doing DFS for my route first and then DFS for the elephant's route. Took at most 2 minutes to run. Some details:
  - Whatever valves me and the elephant visit in the optimal route, each of us must take the fastest path from one to the next and we must open disjoint sets of valves.
  - We can find each of our paths by searching the tree of paths, but note that we don't necessarily need to search to the leaves: it's possible my path in isolation could have an extra valve opening tacked on at the end, but the elephant was planning on opening that valve earlier on than me so I should forgo opening that valve. It's not "locally optimal" for just me, but it is optimal when the elephant wants to open it.
  - With those in mind, I can do a DFS for my path and then a DFS for the elephant's path, restricting the elephant to valves I did not open. But for my DFS, initiate an elephant DFS after each of my valve openings in case the best path for me is not "locally optimal".

In [133]:
valves = [parse(line) for line in split(t)]

vertices = {}
names = {}
for i, valve in zip(range(len(valves)), valves):
    vertices[valve[0]] = i
    names[i] = valve[0]

n = len(valves)
A = np.zeros((n,n), dtype=int)
for me, flow, others in valves:
    for other in others:
        A[vertices[me], vertices[other]] = 1

D = shortest_path(A).astype(int)

flows = [flow for me, flow, others in valves]

can_open = [i for i in range(n) if flows[i] > 0]

In [135]:
max_value = 0

def DFS_me(i, t, value, can_open, history):
    global max_value
    
    for j in can_open:
        time_to_open = D[i,j] + 1
        if time_to_open < t:
            new_t = t - time_to_open
            new_value = value + flows[j] * new_t
            new_can_open = [k for k in can_open if k != j]
            
            # Make sure to account for the elephant in the overestimate of total value!
            # i.e. That the elephant could open any of the unopened valves and have it on for 24 minutes
            new_total_value_est = new_value + sum([flows[k] for k in new_can_open]) * 24
            if new_total_value_est > max_value:
                DFS_me(j, new_t, new_value, new_can_open, history + [names[j]])
    
    # Now have the elephant plan its route
    # (Since it's possible I don't "complete" my route, and I leave a valve closed for the elephant to open in the past)
    DFS_elephant(vertices['AA'], 26, value, can_open, history + [''])

def DFS_elephant(i, t, value, can_open, history):
    global max_value, max_history
    
    # Flag for whether the elephant is done opening valves
    # Flip to false if we end up opening a valve from here
    done = True
    
    for j in can_open:
        time_to_open = D[i,j] + 1
        if time_to_open < t:
            new_t = t - time_to_open
            new_value = value + flows[j] * new_t
            new_can_open = [k for k in can_open if k != j]
            
            new_total_value_est = new_value + sum([flows[k] for k in new_can_open]) * (new_t - 2)
            if new_total_value_est > max_value:
                done = False
                DFS_elephant(j, new_t, new_value, new_can_open, history + [names[j]])
    
    # If we opened no valves, the elephant's route is done
    # Check if we improved on the best route so far
    if done and value > max_value:
        max_value = value
        max_history = history
        print(max_value)

DFS_me(vertices['AA'], 26, 0, can_open, [])
print(max_value)
print(max_history)

1244
1601
1623
1643
1644
1653
1655
1656
1696
1704
1705
1706
1707
1707
['DD', 'HH', 'EE', '', 'JJ', 'BB', 'CC']


## 2 run

In [136]:
valves = [parse(line) for line in split(s)]

vertices = {}
names = {}
for i, valve in zip(range(len(valves)), valves):
    vertices[valve[0]] = i
    names[i] = valve[0]

n = len(valves)
A = np.zeros((n,n), dtype=int)
for me, flow, others in valves:
    for other in others:
        A[vertices[me], vertices[other]] = 1

D = shortest_path(A).astype(int)

flows = [flow for me, flow, others in valves]

can_open = [i for i in range(n) if flows[i] > 0]

In [137]:
max_value = 0

def DFS_me(i, t, value, can_open, history):
    global max_value
    
    for j in can_open:
        time_to_open = D[i,j] + 1
        if time_to_open < t:
            new_t = t - time_to_open
            new_value = value + flows[j] * new_t
            new_can_open = [k for k in can_open if k != j]
            
            # Make sure to account for the elephant in the overestimate of total value!
            # i.e. That the elephant could open any of the unopened valves and have it on for 24 minutes
            new_total_value_est = new_value + sum([flows[k] for k in new_can_open]) * 24
            if new_total_value_est > max_value:
                DFS_me(j, new_t, new_value, new_can_open, history + [names[j]])
    
    # Now have the elephant plan its route
    # (Since it's possible I don't "complete" my route, and I leave a valve closed for the elephant to open in the past)
    DFS_elephant(vertices['AA'], 26, value, can_open, history + [''])

def DFS_elephant(i, t, value, can_open, history):
    global max_value, max_history
    
    # Flag for whether the elephant is done opening valves
    # Flip to false if we end up opening a valve from here
    done = True
    
    for j in can_open:
        time_to_open = D[i,j] + 1
        if time_to_open < t:
            new_t = t - time_to_open
            new_value = value + flows[j] * new_t
            new_can_open = [k for k in can_open if k != j]
            
            new_total_value_est = new_value + sum([flows[k] for k in new_can_open]) * (new_t - 2)
            if new_total_value_est > max_value:
                done = False
                DFS_elephant(j, new_t, new_value, new_can_open, history + [names[j]])
    
    # If we opened no valves, the elephant's route is done
    # Check if we improved on the best route so far
    if done and value > max_value:
        max_value = value
        max_history = history
        print(max_value)

DFS_me(vertices['AA'], 26, 0, can_open, [])
print(max_value)
print(max_history)

1270
1294
1307
1324
1329
1344
1402
1407
1442
1478
1480
1508
1514
1516
1542
1544
1554
1576
1580
1594
1624
1633
1639
1646
1661
1665
1679
1709
1718
1724
1731
1732
1736
1750
1780
1789
1795
1802
1861
1866
1868
1874
1897
1905
1910
1912
1917
1919
1921
1932
1945
1959
1963
1971
1977
1981
1989
1990
1994
2006
2031
2033
2047
2059
2067
2070
2095
2097
2111
2123
2131
2166
2176
2230
2240
2276
2286
2288
2293
2316
2325
2333
2343
2344
2395
2405
2419
2435
2446
2454
2467
2469
2509
2526
2536
2536
['HR', 'DW', 'XO', 'VI', 'MW', 'FQ', '', 'MA', 'II', 'AS', 'RU', 'PM', 'KQ']


# Utilities

In [1]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s, block_char = '\n\n', line_char = '\n'):
    out = [block.split(line_char) for block in clean(s).split(block_char)]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

# Inputs

In [2]:
t = """
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
...
Valve JJ has flow rate=21; tunnel leads to valve II
"""

In [3]:
s = """
Valve WT has flow rate=0; tunnels lead to valves BD, FQ
...
Valve FQ has flow rate=12; tunnels lead to valves QN, WT, UG, RQ, QM
"""